## Training neural networks

This notebook describes the workflow for training neural networks

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from pathlib import Path
from functools import reduce

import yaml
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from dstools.reproducibility import make_filename

from yass import util

Using TensorFlow backend.


In [4]:
# for reference
print('YASS version is: {}'.format(util.get_version()))

YASS version is: fe1bd86 updates detect.run to use new separated code


In [5]:
path_to_data = Path('~', 'data').expanduser()
path_to_models = Path(path_to_data, 'models')

## Loading training data

In [12]:
path_to_sets = Path(path_to_data, 'training-sets')

path_to_x = Path(path_to_sets, 'x-triage-31wf7ch-15-Aug-2018@00-14-33.npy')
path_to_y = Path(path_to_sets, 'y-triage-31wf7ch-15-Aug-2018@00-14-33.npy')

x_triage = np.load(path_to_x)
y_triage = np.load(path_to_y)

In [13]:
def make_model(x_train, input_shape):
    n_data, window_size, n_channels, _ = x_train.shape

    model = Sequential()
        
#     model.add(MaxPooling2D(pool_size=(2, 1), data_format="channels_last", padding='same'))
    
#     model.add(Dropout(0.75))

    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last", input_shape=input_shape))


#     model.add(Conv2D(70, kernel_size=(window_size, 1),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='relu', use_bias=True,
                     data_format="channels_last"))


#     model.add(Conv2D(70, kernel_size=(1, n_channels),
#                      padding='valid', activation='relu', use_bias=True,
#                      data_format="channels_last"))
    
    model.add(Conv2D(10, kernel_size=(5, 5),
                     padding='same', activation='linear', use_bias=True,
                     data_format="channels_last"))
        
#     model.add(MaxPooling2D(pool_size=(1, 2), data_format="channels_last", padding='same'))
#     model.add(Dropout(0.75))
    
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    
    model.summary()
    
    # initiate RMSprop optimizer
    opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
    opt = keras.optimizers.adam(lr=0.001)

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    return model


# Triage Ttaining

In [14]:
# import models
x = x_triage[:, : , :, np.newaxis]

(x_train, x_test,
y_train, y_test) = train_test_split(x, y_triage, test_size=0.3)

_, wf, ch, _ = x_train.shape

m = make_model(x, (wf, ch, 1))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 31, 7, 10)         260       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 31, 7, 10)         2510      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 31, 7, 10)         2510      
_________________________________________________________________
flatten_2 (Flatten)          (None, 2170)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 2171      
Total params: 7,451
Trainable params: 7,451
Non-trainable params: 0
_________________________________________________________________


In [18]:
m.fit(x_train, y_train,
          batch_size=10000, epochs=100, shuffle=True,
          validation_data=(x_test, y_test))

Train on 28224 samples, validate on 12096 samples
Epoch 1/100
28224/28224 [==============================] - 1s 30us/step - loss: 0.1639 - acc: 0.9378 - val_loss: 0.1604 - val_acc: 0.9353
Epoch 2/100
28224/28224 [==============================] - 1s 27us/step - loss: 0.1646 - acc: 0.9363 - val_loss: 0.1581 - val_acc: 0.9365
Epoch 3/100
28224/28224 [==============================] - 1s 27us/step - loss: 0.1614 - acc: 0.9386 - val_loss: 0.1547 - val_acc: 0.9383
Epoch 4/100
28224/28224 [==============================] - 1s 27us/step - loss: 0.1610 - acc: 0.9379 - val_loss: 0.1538 - val_acc: 0.9403
Epoch 5/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1572 - acc: 0.9401 - val_loss: 0.1560 - val_acc: 0.9396
Epoch 6/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1571 - acc: 0.9398 - val_loss: 0.1508 - val_acc: 0.9411
Epoch 7/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1541 - acc: 0.9412 - val_loss: 0.1496 - 

28224/28224 [==============================] - 1s 26us/step - loss: 0.1159 - acc: 0.9554 - val_loss: 0.1168 - val_acc: 0.9540
Epoch 60/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1152 - acc: 0.9554 - val_loss: 0.1152 - val_acc: 0.9545
Epoch 61/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1154 - acc: 0.9560 - val_loss: 0.1147 - val_acc: 0.9550
Epoch 62/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1139 - acc: 0.9561 - val_loss: 0.1151 - val_acc: 0.9552
Epoch 63/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1133 - acc: 0.9568 - val_loss: 0.1150 - val_acc: 0.9555
Epoch 64/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1134 - acc: 0.9565 - val_loss: 0.1146 - val_acc: 0.9546
Epoch 65/100
28224/28224 [==============================] - 1s 26us/step - loss: 0.1147 - acc: 0.9559 - val_loss: 0.1141 - val_acc: 0.9544
Epoch 66/100
28224/28224 [==============

In [19]:
_, wf, ch, _ = m.input_shape

prefix = f'triage-{wf}wf{ch}ch-'
name_model, name_metadata = make_filename(prefix=prefix, extension=('h5', 'yaml'))
print(name_model)
print(name_metadata)

triage-31wf7ch-15-Aug-2018@00-17-16.h5
triage-31wf7ch-15-Aug-2018@00-17-16.yaml


In [20]:
path_to_model = Path(path_to_models, name_model)
path_to_metadata = Path(path_to_models, name_metadata)

m.save(path_to_model)

metadata = {'path_to_x': str(path_to_x), 'path_to_y': str(path_to_x), 'input_shape': list(m.input_shape)}

with open(path_to_metadata, 'w') as f:
    yaml.dump(metadata, f)